In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.set_option("display.max_columns", None)


In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
train_df.head(10)

In [ ]:
test_df.head(10)

So, there are few features and one target variable, of course ID is of not use so first lets get rid of it

In [ ]:
train_df = train_df.drop(["id"], axis = 1)
test_df = test_df.drop(["id"], axis = 1)

In [ ]:
Summary = pd.DataFrame(train_df.dtypes, columns=["dtype"])
Summary["max"] = train_df.max()
Summary["min"] = train_df.min()
Summary["null"] = train_df.isnull().sum()
Summary["first"] = train_df.iloc[0]
Summary["second"] = train_df.iloc[1]
Summary["third"] = train_df.iloc[2]
Summary

In [ ]:
len(train_df) #300000

In [ ]:

cm = train_df.corr()
fig, ax = plt.subplots(figsize=(15,10))  
sns.heatmap(cm, annot = True, vmin=-1, vmax=1, center= 0,cmap= 'coolwarm', ax = ax)

In [ ]:
correlated_features = set()
for i in range(len(cm.columns)):
    for j in range(i):
        if abs(cm.iloc[i, j]) > 0.78:
            colname = cm.columns[i]
            print(colname)
            correlated_features.add(colname)

In [ ]:
# # gettig rid of these features
# train_df = train_df.drop(correlated_features, axis = 1)
# test_df = test_df.drop(correlated_features, axis = 1)

There is not much diff in the max value and the min value so I do not think there is a need to scale

In [ ]:
y = train_df["target"]
train_df.drop(["target"], axis = 1, inplace = True)

### Adding more features

In [ ]:
# train_df["max"] = train_df.max(axis = 1)
# train_df["min"] = train_df.min(axis = 1)
# train_df["skew"] = train_df.skew(axis = 1)
# train_df["mean"] = train_df.mean(axis = 1)
# train_df["median"] = train_df.median(axis = 1)

In [ ]:
# test_df["max"] = test_df.max(axis = 1)
# test_df["min"] = test_df.min(axis = 1)
# test_df["skew"] = test_df.skew(axis = 1)
# test_df["mean"] = test_df.mean(axis = 1)
# test_df["median"] = test_df.median(axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


In [ ]:
len(train_df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size = 0.2, random_state = 123)

In [ ]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test= scaler.transform(X_test)
# test_df= scaler.transform(test_df)

In [ ]:
# lm = LinearRegression()
# lm.fit(X_train, y_train)

In [ ]:
# y_pred = lm.predict(X_test)
# loss = mean_squared_error(y_pred, y_test )

In [ ]:
# np.sqrt(loss)
# End goal
# Trynna to reduce rmse.
# 0.725037588861915 without doing anything
# with more featues it did help  to a bit 0.7246819980764818
# with scaling 0.7246819980764818, it did not help much 0.7246905107733411
# withe removing correlated features, it did not help, 0.7250850271199127

In [ ]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
# y_res = lm.predict(test_df)

In [ ]:
# rr = RandomForestRegressor()
# rr.fit(X_train, y_train)
# y_pred = rr.predict(X_test)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
# XGB
# model = XGBRegressor(objective='reg:squarederror',
#                      booster = "gbtree",
#                      eval_metric = "rmse",
#                      tree_method = "gpu_hist",
#                      n_estimators = 1000,
#                      learning_rate = 0.04,
#                      eta = 0.1,
#                      max_depth = 7,
#                      subsample=0.85,
#                      colsample_bytree = 0.85,
#                      colsample_bylevel = 0.8,
#                      alpha = 0,
#                      random_state = 2021)


regressor =  XGBRegressor(
                 objective='reg:squarederror',
                 booster = "gbtree",
                 eval_metric = "rmse",
                 tree_method = "gpu_hist",
                 colsample_bytree=0.5,
                 alpha=0.01563,
                 random_state=2021,
                 #gamma=0.0,
                 learning_rate=0.01,
                 max_depth=15,
                 min_child_weight=257,
                 n_estimators=4000,                                                                  
                 #reg_alpha=0.9,
                 reg_lambda=0.003,
                 subsample=0.7,
                 metric_period=100,
                 silent=1)

regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=1)

In [ ]:
# Fit mode
# %time regressor.fit(X_train, y_train)

In [ ]:
y_val_pred = regressor.predict(X_test)
print('Validation Set RMSE:', np.sqrt(mean_squared_error(y_test, y_val_pred)))
# gave the RMSE: 0.6978898866785791
# gave the RMSE without extra features: 0.6973184362827547
# gave the RMSE with hyperparameters tuned: 0.6958703143319863

In [ ]:
y_res_xgb = regressor.predict(test_df)

In [ ]:
# loss = mean_squared_error(y_pred, y_test )
# print(np.sqrt(loss))
# # imporvement with evrerything: 0.7067061724711653 but the prvate score of 0.71153 which ain't that bad but it is still bad 
# y_res1 = rr.predict(test_df)

In [ ]:
y_res_xgb # This made rank to reach in 500. Nice improvement

In [ ]:
sub["target"] = y_res_xgb
sub.to_csv("rc2.csv", index=False)
